In [ ]:
import requests
import csv
import os
from bs4 import BeautifulSoup
url = 'https://www.wunderground.com/dashboard/pws/INEWDE9/table/%d/%d/daily'
h= 'https://www.wunderground.com/history/airport/VIDP/%d/%d/%d/DailyHistory.html'


day_range = list(range(1, 32)) # days, 1 to 31

# months, Aug to Dec for 2020, and Jan for 2021
month_range = {
				2022: list(range(10, 13)),
				2023: list(range(1,3))
				}

year_range = [2022, 2023]

if not os.path.exists('Weather_Data'):
    os.makedirs('Weather_Data')


for year in year_range:
    for month in month_range[year]:
        month_dir = 'Weather_Data/%d/%02d/'%(year, month)
        if not os.path.exists(month_dir):os.makedirs(month_dir)
        for day in day_range:
            try:
                date = '%02d/%02d/%d' %(day, month, year)
                print('Scraping', date)

                current_url = url % (year, month, day)
                resp = requests.get(current_url, timeout=(3.05, 27)) # send a get request to the url, get response
                soup = BeautifulSoup(resp.text, 'lxml') # Yummy HTML soup
                # print(soup)
                table = soup.find('table', {'id':'obsTable'}) # get the table from html
                trs = table.findAll('tr') # extract all rows of the table
                if len(trs[1:])!=0:
                    csv_filename = month_dir + '%s.csv' % date.replace('/', '-')
                    if os.path.exists(csv_filename): os.remove(csv_filename) # remove the file it already exists, can result in data duplicacy
                    with open(csv_filename, 'a') as f:
                        writer = csv.writer(f)
                        columns = [th.text for th in trs[0].findChildren('th')]					
                        writer.writerow(columns)
                        for tr in trs[1:]:
                            row = []
                            tds = tr.findChildren('td')
                            for td in tds:
                                span = td.findChildren('span', {'class':'wx-value'})
                                if span:
                                    row.append(span[0].text.strip())
                                else:
                                  row.append(td.text.strip())
                            
                            assert len(row) == len(columns)
                            writer.writerow(row)
                  
            except Exception as e:
                print('Exception', e)
                print(date)
                print(current_url)

Scraping 01/10/2022
Exception 'NoneType' object has no attribute 'findAll'
01/10/2022
https://www.wunderground.com/history/airport/VIDP/2022/10/1/DailyHistory.html
Scraping 02/10/2022
Exception 'NoneType' object has no attribute 'findAll'
02/10/2022
https://www.wunderground.com/history/airport/VIDP/2022/10/2/DailyHistory.html
Scraping 03/10/2022
Exception 'NoneType' object has no attribute 'findAll'
03/10/2022
https://www.wunderground.com/history/airport/VIDP/2022/10/3/DailyHistory.html
Scraping 04/10/2022
Exception 'NoneType' object has no attribute 'findAll'
04/10/2022
https://www.wunderground.com/history/airport/VIDP/2022/10/4/DailyHistory.html
Scraping 05/10/2022
Exception 'NoneType' object has no attribute 'findAll'
05/10/2022
https://www.wunderground.com/history/airport/VIDP/2022/10/5/DailyHistory.html
Scraping 06/10/2022
Exception 'NoneType' object has no attribute 'findAll'
06/10/2022
https://www.wunderground.com/history/airport/VIDP/2022/10/6/DailyHistory.html
Scraping 07/10/2

KeyboardInterrupt: ignored

In [ ]:
freq = '5min'


if freq == '5min':
        timespan = 'daily'
# the url for daily summary data (avg/min/max) is called "monthly" on wunderground
elif freq == 'daily':
    timespan = 'monthly'

# Render the url and open the page source as BS object
url = 'https://www.wunderground.com/dashboard/pws/%s/table/%s/%s/%s' % (station,
                                                                        date, date, 
                                                                        timespan)
r = render_page(url)
soup = BS(r, "html.parser",)

container = soup.find('lib-history-table')

# Check that lib-history-table is found
if container is None:
    raise ValueError("could not find lib-history-table in html source for %s" % url)

# Get the timestamps and data from two separate 'tbody' tags
all_checks = container.find_all('tbody')
time_check = all_checks[0]
data_check = all_checks[1]

# Iterate through 'tr' tags and get the timestamps
hours = []
for i in time_check.find_all('tr'):
    trial = i.get_text()
    hours.append(trial)

# For data, locate both value and no-value ("--") classes
classes = ['wu-value wu-value-to', 'wu-unit-no-value ng-star-inserted']

# Iterate through span tags and get data
data = []
for i in data_check.find_all('span', class_=classes):
    trial = i.get_text()
    data.append(trial)

columns = {'5min': ['Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 
                    'Wind Gust', 'Pressure', 'Precip. Rate', 'Precip. Accum.'],
            'daily': ['Temperature_High', 'Temperature_Avg', 'Temperature_Low', 
                      'DewPoint_High', 'DewPoint_Avg', 'DewPoint_Low', 
                      'Humidity_High', 'Humidity_Avg', 'Humidity_Low', 
                      'WindSpeed_High', 'WindSpeed_Avg', 'WindSpeed_Low', 
                      'Pressure_High', 'Pressure_Low', 'Precip_Sum']}

# Convert NaN values (stings of '--') to np.nan
data_nan = [np.nan if x == '--' else x for x in data]

# Convert list of data to an array
data_array = np.array(data_nan, dtype=float)
data_array = data_array.reshape(-1, len(columns[freq]))

# Prepend date to HH:MM strings
if freq == '5min':
    timestamps = ['%s %s' % (date, t) for t in hours]
else:
    timestamps = hours

# Convert to dataframe
df = pd.DataFrame(index=timestamps, data=data_array, columns=columns[freq])
df.index = pd.to_datetime(df.index)

return df

In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.8 MB/s eta 0:00:00


In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

from selenium import webdriver
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu focal InRelease
Get:4 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ Packages [76.4 kB]
Get:7 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  Packages [993 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:12 http://ppa.launchpad.net/

<ipython-input-5-d40b5adfff85>:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)


WebDriverException: ignored

In [ ]:
%%shell
# Ubuntu no longer distributes chromium-browser outside of snap
#
# Proposed solution: https://askubuntu.com/questions/1204571/how-to-install-chromium-without-snap

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF

# Install chromium and chromium-driver
apt-get update
apt-get install chromium chromium-driver

# Install selenium
pip install selenium

Executing: /tmp/apt-key-gpghome.wpXzQ5Qvfk/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.7YNtTBnuPG/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ksLrhAzlB4/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Hit:2 http://archive.ubuntu.com/ubuntu 

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")

options.headless = True

driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


<ipython-input-2-5525c108f2db>:9: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True
<ipython-input-2-5525c108f2db>:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)


In [ ]:
# -*- coding: utf-8 -*-

"""Module to scrape 5-min personal weather station data from Weather Underground.
Usage is:
>>> python scrape_wunderground.py   STATION    DATE     FREQ
where station is a personal weather station (e.g., KCAJAMES3), date is in the 
format YYYY-MM-DD and FREQ is either 'daily' or '5min' (for daily or 5-minute
observations, respectively). 
Alternatively, each function below can be imported and used in a separate python
script. Note that a working version of chromedriver must be installed and the absolute 
path to executable has to be updated below ("chromedriver_path").
Zach Perzan, 2021-07-28"""

import time
import sys

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup as BS
from selenium import webdriver



In [ ]:
# Set the absolute path to chromedriver
chromedriver_path = '/bin/chromedriver'




In [ ]:
def render_page(url):
    """Given a url, render it with chromedriver and return the html source
    
    Parameters
    ----------
        url : str
            url to render
    
    Returns
    -------
        r : 
            rendered page source
    """
    
    driver = webdriver.Chrome(chromedriver_path)
    driver.get(url)
    time.sleep(3) # Could potentially decrease the sleep time
    r = driver.page_source
    driver.quit()

    return r




In [ ]:
def scrape_wunderground(station, date, freq='5min'):
    """Given a PWS station ID and date, scrape that day's data from Weather 
    Underground and return it as a dataframe.
    
    Parameters
    ----------
        station : str
            The personal weather station ID
        date : str
            The date for which to acquire data, formatted as 'YYYY-MM-DD'
        freq : {'5min', 'daily'}
            Whether to download 5-minute weather observations or daily 
            summaries (average, min and max for each day)
            
    Returns
    -------
        df : dataframe or None
            A dataframe of weather observations, with index as pd.DateTimeIndex 
            and columns as the observed data
    """
    
    # the url for 5-min data is called "daily" on weather underground
    if freq == '5min':
        timespan = 'daily'
    # the url for daily summary data (avg/min/max) is called "monthly" on wunderground
    elif freq == 'daily':
        timespan = 'monthly'
    
    # Render the url and open the page source as BS object
    url = 'https://www.wunderground.com/dashboard/pws/%s/table/%s/%s/%s' % (station,
                                                                            date, date, 
                                                                            timespan)
    r = render_page(url)
    soup = BS(r, "html.parser",)

    container = soup.find('lib-history-table')
    
    # Check that lib-history-table is found
    if container is None:
        print("error")
        raise ValueError("could not find lib-history-table in html source for %s" % url)
    
    # Get the timestamps and data from two separate 'tbody' tags
    all_checks = container.find_all('tbody')
    time_check = all_checks[0]
    data_check = all_checks[1]

    # Iterate through 'tr' tags and get the timestamps
    hours = []
    for i in time_check.find_all('tr'):
        trial = i.get_text()
        hours.append(trial)

    # For data, locate both value and no-value ("--") classes
    classes = ['wu-value wu-value-to', 'wu-unit-no-value ng-star-inserted']

    # Iterate through span tags and get data
    data = []
    for i in data_check.find_all('span', class_=classes):
        trial = i.get_text()
        data.append(trial)

    columns = {'5min': ['Temperature', 'Dew Point', 'Humidity', 'Wind Speed', 
                        'Wind Gust', 'Pressure', 'Precip. Rate', 'Precip. Accum.'],
               'daily': ['Temperature_High', 'Temperature_Avg', 'Temperature_Low', 
                         'DewPoint_High', 'DewPoint_Avg', 'DewPoint_Low', 
                         'Humidity_High', 'Humidity_Avg', 'Humidity_Low', 
                         'WindSpeed_High', 'WindSpeed_Avg', 'WindSpeed_Low', 
                         'Pressure_High', 'Pressure_Low', 'Precip_Sum']}

    # Convert NaN values (stings of '--') to np.nan
    data_nan = [np.nan if x == '--' else x for x in data]

    # Convert list of data to an array
    data_array = np.array(data_nan, dtype=float)
    data_array = data_array.reshape(-1, len(columns[freq]))

    # Prepend date to HH:MM strings
    if freq == '5min':
        timestamps = ['%s %s' % (date, t) for t in hours]
    else:
        timestamps = hours

    # Convert to dataframe
    df = pd.DataFrame(index=timestamps, data=data_array, columns=columns[freq])
    df.index = pd.to_datetime(df.index)
    
    return df








In [ ]:
def scrape_multiattempt(station, date, attempts=4, wait_time=5.0, freq='5min'):
    """Try to scrape data from Weather Underground. If there is an error on the 
    first attempt, try again.
    
    Parameters
    ----------
        station : str
            The personal weather station ID
        date : str
            The date for which to acquire data, formatted as 'YYYY-MM-DD'
        attempts : int, default 4
            Maximum number of times to try accessing before failuer
        wait_time : float, default 5.0
            Amount of time to wait in between attempts
        freq : {'5min', 'daily'}
            Whether to download 5-minute weather observations or daily 
            summaries (average, min and max for each day)
            
    Returns
    -------
        df : dataframe or None
            A dataframe of weather observations, with index as pd.DateTimeIndex 
            and columns as the observed data
    """
    
    # Try to download data limited number of attempts
    for n in range(attempts):
        try:
            print(station,date)
            df = scrape_wunderground(station, date, freq=freq)
        except:
            # if unsuccessful, pause and retry
            time.sleep(wait_time)
        else: 
            # if successful, then break
            break
    # If all attempts failed, return empty df
    else:
        df = pd.DataFrame()
        
    return df
    


In [ ]:
def scrape_multidate(station, start_date, end_date, freq='5min'):
    """Given a PWS station ID and a start and end date, scrape data from Weather
        Underground for that date range and return it as a dataframe.
        Parameters
        ----------
            station : str
                The personal weather station ID
            start_date : str
                The date for which to begin acquiring data, formatted as 'YYYY-MM-DD'
            end_date : str
                The date for which to end acquiring data, formatted as 'YYYY-MM-DD'
        Returns
        -------
            df : dataframe or None
                A dataframe of weather observations, with index as pd.DateTimeIndex
                and columns as the observed data
        """
    # Convert end_date and start_date to datetime types
    end_date = datetime.strptime(end_date, '%Y-%m-%d').date()
    start_date = datetime.strptime(start_date, '%Y-%m-%d').date()

    # Calculate time delta
    delta = end_date - start_date

    # Create list dates and append all days within the start and end date to dates
    dates = []
    for i in range(delta.days + 1):
        day = start_date + timedelta(days=i)
        dates.append(day)
    dates = [date.strftime('%Y-%m-%d') for date in dates]

    # Repeat the station name in a list for as many dates are in the date range
    stations = [station] * len(dates)

    # Scrape wunderground for data from all dates in range and store in list of dateframes
    df_list = list(map(scrape_multiattempt, stations, dates, freq=freq))

    # Convert list of dataframes to one dataframe
    df = pd.concat(df_list)

    return df

In [ ]:
if __name__ == "__main__":
    
    station = 'INEWDE9'
    date = '2023-03-05'
    
    df = scrape_multiattempt(station, date)
    
    filename = '%s_%s.csv' % (station, date)
    df.to_csv(filename)

INEWDE9 2023-03-05
INEWDE9 2023-03-05
INEWDE9 2023-03-05
INEWDE9 2023-03-05
